In [4]:
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from glob import glob
import os

In [5]:
LABEL_INT_DICT = np.unique(pd.read_csv('ucf11_train_vid.csv')['label'])
LABEL_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_INT_DICT)
pprint(LABEL_STR_DICT)

array(['biking', 'diving', 'golf', 'juggle', 'jumping', 'riding',
       'shooting', 'spiking', 'swing', 'tennis', 'walk'], dtype=object)
{'biking': 0,
 'diving': 1,
 'golf': 2,
 'juggle': 3,
 'jumping': 4,
 'riding': 5,
 'shooting': 6,
 'spiking': 7,
 'swing': 8,
 'tennis': 9,
 'walk': 10}


In [7]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

MAX_FRAMES = 10
NUM_FEATURES = 1280

def build_model():
    inputs = keras.Input((MAX_FRAMES, NUM_FEATURES))
    x = keras.layers.LSTM(64, return_sequences=True)(
        inputs
    )
    x = keras.layers.LSTM(64, return_sequences=False)(
        x
    )
    x = keras.layers.Dropout(0.3)(x)
    outputs = keras.layers.Dense(len(LABEL_INT_DICT), activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    return model

adam = keras.optimizers.Adam(lr=0.0001)
model = build_model()

model.compile(
    optimizer=adam,
    loss = 'sparse_categorical_crossentropy',
    metrics='accuracy'
)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 1280)]        0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 64)            344320    
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 11)                715       
                                                                 
Total params: 378,059
Trainable params: 378,059
Non-trainable params: 0
_________________________________________________________________


In [10]:
import albumentations as A
import cv2
import math
import tensorflow as tf
from tensorflow import keras

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size,
        mode='train', shuffle=True):
        assert mode in ['train', 'valid']
        self.mode = mode
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.npy_paths = glob(
            f'UCF11_updated_npy/{mode}/*.npy'
        )
    
    def __len__(self):
        return math.ceil(len(self.npy_paths) / self.batch_size)
    
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.npy_paths[strt:fin]
    
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
    
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for npy_path in data:
            npy = np.load(npy_path)
            
            label = npy_path.split('\\')[-1].split('_')[0]
            label = LABEL_STR_DICT[label]
            
            batch_x.append(npy)
            batch_y.append(label)
            
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.npy_paths)
    
train_generator = DataGenerator(
    mode='train',
    batch_size=128,
    shuffle=True)

valid_generator = DataGenerator(
    mode='valid',
    batch_size=128,
    shuffle=True)

In [11]:
for x, y in train_generator:
    print(x.shape)
    print(y.shape)
    break

(128, 10, 1280)
(128,)


In [13]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    verbose=1
)

Epoch 1/30
2/2 [==============================] - 0s 137ms/step - loss: 1.7597 - accuracy: 0.8455 - val_loss: 1.7883 - val_accuracy: 0.8909
Epoch 2/30
2/2 [==============================] - 0s 131ms/step - loss: 1.6650 - accuracy: 0.9000 - val_loss: 1.7416 - val_accuracy: 0.8909
Epoch 3/30
2/2 [==============================] - 0s 123ms/step - loss: 1.6370 - accuracy: 0.8864 - val_loss: 1.6957 - val_accuracy: 0.8909
Epoch 4/30
2/2 [==============================] - 0s 120ms/step - loss: 1.5905 - accuracy: 0.9045 - val_loss: 1.6507 - val_accuracy: 0.8909
Epoch 5/30
2/2 [==============================] - 0s 139ms/step - loss: 1.5378 - accuracy: 0.9227 - val_loss: 1.6065 - val_accuracy: 0.8909
Epoch 6/30
2/2 [==============================] - 0s 122ms/step - loss: 1.4939 - accuracy: 0.9727 - val_loss: 1.5632 - val_accuracy: 0.9091
Epoch 7/30
2/2 [==============================] - 0s 105ms/step - loss: 1.4317 - accuracy: 0.9545 - val_loss: 1.5207 - val_accuracy: 0.8909
Epoch 8/30
2/2 [====